In [ ]:
%pylab inline

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
df = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/train.csv', index_col=0)
df_test = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/test.csv', index_col=0)
df_sub  = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/sample_submission.csv', index_col=0)

In [ ]:
n_classes = len( df.target.value_counts() )
n_classes

In [ ]:
from sklearn.preprocessing import LabelBinarizer, LabelEncoder

In [ ]:
import xgboost as xgb

tgt_name = 'target'

y = df[tgt_name]
X = df.drop(tgt_name, axis=1)

label_encoder = LabelEncoder()
_y = label_encoder.fit_transform(y)

for _i, _c in enumerate(label_encoder.classes_):
    print(_i, _c)

In [ ]:
_X_train = xgb.DMatrix(X, _y)

In [ ]:
params = {
    'objective': 'multi:softprob',
    'tree_method': 'gpu_hist',
    'eval_metric': 'mlogloss',
    'max_depth': 3,
    'colsample_bytree': 0.1,
    'lambda': 10,
    'alpha': 10,
    'subsample': 0.1,
    'eta': 0.05,
    'num_class': n_classes
}

history = xgb.cv(params, _X_train, 1000, nfold=5, stratified=True)

figure(figsize=(3,3))
history.plot(y=['train-mlogloss-mean', 'test-mlogloss-mean'])
ylim(1.7,1.8)
grid()
history

In [ ]:
X_train = xgb.DMatrix(X, _y)
X_test  = xgb.DMatrix(df_test)
model = xgb.train(params, X_train, 1000, verbose_eval=10, evals=[(X_train, 'train')])

In [ ]:
model.predict(X_test)
df_sub.loc[:,:] = model.predict(X_test)
df_sub.to_csv('submission.csv')